# 05x Tools - Prediction

## WORK IN PROGRESS (7/28/2022)
Predictions from models created in the 05 series of notebooks.


This notebook will feature example of
- using an endpoint for predictions: sync and async
    - demonstrate endpoint scaling (async)
- using traffic split for multiple models on an endpoint
- hosting multiple models on an endpoint
- create batch prediction jobs
- import model into BigQuery for Batch Prediction

---
## Setup

inputs:

In [87]:
project = !gcloud config get-value project
PROJECT_ID = project[0]
PROJECT_ID

'statmike-mlops-349915'

In [88]:
REGION = 'us-central1'
DATANAME = 'fraud'
SERIES = '05'

# Model Training
VAR_TARGET = 'Class'
VAR_OMIT = 'transaction_id' # add more variables to the string with space delimiters
EPOCHS = 10
BATCH_SIZE = 100

packages:

In [89]:
from google.cloud import aiplatform

from google.cloud import bigquery
from google.protobuf import json_format
from google.protobuf.struct_pb2 import Value
import json
import numpy as np

clients:

In [90]:
aiplatform.init(project=PROJECT_ID, location=REGION)
bq = bigquery.Client()

parameters:

environment:

---
## Online Predictions: Synchronous Examples

### Get Endpoint

```python
endpoint
endpoint.display_name
endpoint.resource_name
endpoint.traffic_split
endpoint.list_models()
```

In [91]:
endpoints = aiplatform.Endpoint.list(filter = f"display_name={SERIES}_{DATANAME}")
endpoint = endpoints[0]

### Retrieve records for prediction

In [92]:
pred = bq.query(query = f"SELECT * FROM {DATANAME}.{DATANAME}_prepped WHERE splits='TEST' LIMIT 1000").to_dataframe()

In [93]:
pred.head(4)

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V23,V24,V25,V26,V27,V28,Amount,Class,transaction_id,splits
0,32799,1.153477,-0.047859,1.358363,1.480620,-1.222598,-0.481690,-0.654461,0.128115,0.907095,...,-0.025964,0.701843,0.417245,-0.257691,0.060115,0.035332,0.0,0,e9d16028-4b41-4753-87ee-041d33642ae9,TEST
1,35483,1.286640,0.072917,0.212182,-0.269732,-0.283961,-0.663306,-0.016385,-0.120297,-0.135962,...,0.052674,0.076792,0.209208,0.847617,-0.086559,-0.008262,0.0,0,8b319d3a-2b2d-445b-a9a2-0da3d664ec2a,TEST
2,163935,1.961967,-0.247295,-1.751841,-0.268689,0.956431,0.707211,0.020675,0.189433,0.455055,...,0.186420,-1.621368,-0.131098,0.034276,-0.004909,-0.090859,0.0,0,788afb87-60aa-4482-8b48-c924bec634aa,TEST
3,30707,-0.964364,0.176372,2.464128,2.672539,0.145676,-0.152913,-0.591983,0.305066,-0.148034,...,-0.024200,0.365226,-0.745369,-0.060544,0.095692,0.217639,0.0,0,473d0936-1974-4ae8-ab70-230e7599bd3f,TEST


In [94]:
newobs = pred[pred.columns[~pred.columns.isin(VAR_OMIT.split()+[VAR_TARGET, 'splits'])]].to_dict(orient='records')
#newobs[0]

In [95]:
len(newobs)

1000

### Get Single Prediction: Python Client

In [96]:
instances = [json_format.ParseDict(newobs[0], Value())]
parameters = json_format.ParseDict({}, Value())

prediction = endpoint.predict(instances=instances, parameters=parameters)
prediction

Prediction(predictions=[[0.999275744, 0.000724321057]], deployed_model_id='7271945605738397696', model_version_id='', model_resource_name='projects/1026793852137/locations/us-central1/models/model_05i_fraud', explanations=None)

In [97]:
prediction.predictions[0]

[0.999275744, 0.000724321057]

In [98]:
np.argmax(prediction.predictions[0])

0

### Get Synchronous Predictions: Python Client

In [99]:
predictions = []
for p in range(len(newobs)):
    instances = [json_format.ParseDict(newobs[p], Value())]
    parameters = json_format.ParseDict({}, Value())
    prediction = endpoint.predict(instances=instances, parameters=parameters)
    predictions.append(np.argmax(prediction.predictions[0]))

In [100]:
from collections import Counter
c = Counter(predictions)
c

Counter({0: 997, 1: 3})

In [68]:
[i for i, j in enumerate(predictions) if j == 1]

[85, 251, 473]

In [69]:
pred.iloc[[i for i, j in enumerate(predictions) if j == 1]]

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,...,V23,V24,V25,V26,V27,V28,Amount,Class,transaction_id,splits
85,148074,-2.219219,0.727831,-5.458230,5.924850,3.932464,-3.085984,-1.677870,0.865075,-3.177260,...,-0.028752,0.025723,-0.825835,-0.013089,0.413291,-0.131387,0.0,1,a13daf1c-d562-4f9e-b3a4-ba0bfe58d6fa,TEST
251,129668,0.753356,2.284988,-5.164492,3.831112,-0.073622,-1.316596,-1.855495,0.831079,-1.567514,...,0.187697,0.358433,-0.488934,-0.258802,0.296145,-0.047174,2.0,1,ccbcb7b2-79be-4ad0-9b75-771f695ef69b,TEST
473,56887,-0.075483,1.812355,-2.566981,4.127549,-1.628532,-0.805895,-3.390135,1.019353,-2.451251,...,-0.143624,0.013566,0.634203,0.213693,0.773625,0.387434,5.0,1,ba8d853c-0e20-4422-a9f0-03daa3b492e7,TEST


---
## Batch Predictions: BigQuery ML

Load a model to BigQuery and use BQML to create predictions.  Need the URI of TensorFlow model.

### Get Endpoint
Model is currently deployed for online prediction.  Retrieve the current model and import to BigQuery for predictions.

In [70]:
endpoints = aiplatform.Endpoint.list(filter = f"display_name={SERIES}_{DATANAME}")
endpoint = endpoints[0]

In [71]:
endpoint.list_models()[0].model

'projects/1026793852137/locations/us-central1/models/model_05i_fraud'

### Get Model At Endpoint

In [72]:
model = aiplatform.Model(model_name = endpoint.list_models()[0].model)

In [73]:
model.uri

'gs://statmike-mlops-349915/fraud/models/05i/20220728003419/18/model'

In [74]:
model.name

'model_05i_fraud'

### Import Model Into BigQuery

In [81]:
query = f'''
CREATE OR REPLACE MODEL `{PROJECT_ID}.{DATANAME}.{model.name}`
    OPTIONS(
        MODEL_TYPE = 'TENSORFLOW',
        MODEL_PATH = '{model.uri}/*')
'''

In [82]:
print(query)


CREATE OR REPLACE MODEL `statmike-mlops-349915.fraud.model_05i_fraud`
    OPTIONS(
        MODEL_TYPE = 'TENSORFLOW',
        MODEL_PATH = 'gs://statmike-mlops-349915/fraud/models/05i/20220728003419/18/model/*')



In [83]:
job = bq.query(query = query)
job.result()
(job.ended-job.started).total_seconds()

4.049

### Generate Predictions With BigQuery (ML.Predict)

In [86]:
query = f'''
SELECT *
FROM ML.PREDICT(
    MODEL `{PROJECT_ID}.{DATANAME}.{model.name}`, (
        SELECT * 
        FROM {DATANAME}.{DATANAME}_prepped
        WHERE splits='TEST' AND Class = 1
        LIMIT 10
    )
)
'''
results = bq.query(query = query).to_dataframe()
results

,dense,Time,V1,V2,V3,V4,V5,V6,V7,V8,...,V23,V24,V25,V26,V27,V28,Amount,Class,transaction_id,splits
0,"[0.0016647283919155598, 0.9983352422714233]",148074,-2.219219,0.727831,-5.458230,5.924850,3.932464,-3.085984,-1.677870,0.865075,...,-0.028752,0.025723,-0.825835,-0.013089,0.413291,-0.131387,0.0,1,a13daf1c-d562-4f9e-b3a4-ba0bfe58d6fa,TEST
1,"[0.016800928860902786, 0.9831990003585815]",129668,0.753356,2.284988,-5.164492,3.831112,-0.073622,-1.316596,-1.855495,0.831079,...,0.187697,0.358433,-0.488934,-0.258802,0.296145,-0.047174,2.0,1,ccbcb7b2-79be-4ad0-9b75-771f695ef69b,TEST
2,"[0.0005459527601487935, 0.9994540810585022]",56887,-0.075483,1.812355,-2.566981,4.127549,-1.628532,-0.805895,-3.390135,1.019353,...,-0.143624,0.013566,0.634203,0.213693,0.773625,0.387434,5.0,1,ba8d853c-0e20-4422-a9f0-03daa3b492e7,TEST
3,"[0.8952898979187012, 0.10471007227897644]",146998,-2.064240,2.629739,-0.748406,0.694992,0.418178,1.392520,-1.697801,-6.333065,...,0.459861,-1.051685,0.209178,-0.319859,0.015434,-0.050117,8.0,1,f390e04f-14f4-4c77-b18d-6adb8da79b54,TEST
4,"[0.11997880041599274, 0.8800212740898132]",78725,-4.312479,1.886476,-2.338634,-0.475243,-1.185444,-2.112079,-2.122793,0.272565,...,-1.114692,0.269069,-0.020572,-0.963489,-0.918888,0.001454,60.0,1,37db7004-4459-48a3-8464-24c5ff19cc3b,TEST
5,"[4.79785031131931e-16, 0.9999999403953552]",14152,-4.710529,8.636214,-15.496222,10.313349,-4.351341,-3.322689,-10.788373,5.060381,...,0.554408,-1.204042,-0.450685,0.641836,1.605958,0.721644,1.0,1,5bb6c913-1486-4421-a791-f13511b24f1f,TEST
6,"[6.733386092618032e-14, 0.9999999403953552]",21419,-17.467710,10.114816,-24.202142,11.805469,-10.198046,-2.579938,-17.656788,2.256902,...,1.296817,-1.055104,0.111792,0.679695,2.093541,-1.425491,1.0,1,8922139c-c818-4d35-a996-7dc72d19216a,TEST
7,"[1.3924342056270689e-05, 0.9999859929084778]",143456,-2.006582,3.676577,-5.463811,7.232058,-1.627859,-0.996755,-4.299833,2.268867,...,0.167947,-0.449864,0.023702,0.536905,0.485864,-0.042393,1.0,1,ebd4dc94-3c92-41c8-b158-88ea102a5c23,TEST
8,"[1.3694696463062428e-05, 0.9999862909317017]",40662,-4.446847,-0.014793,-5.126307,6.945130,5.269255,-4.297177,-2.591242,0.342671,...,-0.226017,-0.401236,0.856124,0.661272,0.492560,0.971834,1.0,1,632895ce-64c7-4296-a638-ca415be392df,TEST
9,"[0.9690614938735962, 0.030938537791371346]",77171,1.118560,1.291858,-1.298805,2.135772,0.772204,-1.147291,0.390578,-0.107072,...,-0.102326,0.017911,0.650302,-0.332366,0.105949,0.128124,1.0,1,b17e46f7-9352-4bd0-91d4-4260be816363,TEST
